In [3]:
import csv
import json
import pandas as pd
import numpy as np
from pprint import pprint
from collections import defaultdict, Counter
from dateutil.parser import *

SMA_WINDOW = 7

def get_lineitem(d):
    
    caserate_mean = None
    mortality_mean = None
    dailyCases = None
    dailyMortality = None
    hospDaily = None
    percent14dayDailyCases = None
    percent14dayDailyDeaths = None

        
    try:
        caserate_mean = float(d["mean7daycases"])
        mortality_mean = float(d["mean7daydeaths"])
        dailyCases = float(d["dailycases"])
        dailyMortality = float(d["dailydeaths"])
        hospDaily = float(d["hospDaily"])
        percent14dayDailyCases = float(d["percent14dayDailyCases"])
        percent14dayDailyDeaths = float(d["percent14dayDailyDeaths"])
        
    except ValueError:
        caserate_mean  = 0
        mortality_mean = 0
        dailyCases = 0
        dailyMortality = 0
        hospDaily = 0
        percent14dayDailyCases = 0
        percent14dayDailyDeaths = 0
        

    if dailyCases < 0:
        dailyCases = 0
    if dailyMortality < 0:
        dailyMortality = 0
    if caserate_mean < 0:
        caserate_mean = 0
    if mortality_mean < 0:
        mortality_mean = 0
    if hospDaily < 0:
        hospDaily = 0
    if percent14dayDailyCases == -999:
        percent14dayDailyCases = 0
    if percent14dayDailyDeaths == -999:
        percent14dayDailyDeaths = 0


    return {"t": int(parse(d["date"]).timestamp()),
            "caseRateMean": caserate_mean,
            "mortalityMean": mortality_mean,
            "dailyCases": dailyCases,
            "dailyMortality": dailyMortality,
            "hospDaily": hospDaily,
            "percent14dayDailyCases" : percent14dayDailyCases,
            "percent14dayDailyDeaths" : percent14dayDailyDeaths}

def linechart(fn="covidtimeseries.csv"):

    data = defaultdict(list)
    with open(fn, "r") as fp:
        reader = csv.reader(fp)
        header = next(reader)
        for row in reader:
            d = {k:v.strip() for k, v in zip(header, row)}
            
            if d["date"] == "":
                continue

            item = get_lineitem(d)
            k = ""
            if d["nation"] != "":
                k = "_nation"
            elif d["statename"] != "":
                k = d["fips"].zfill(2)
            elif d["county"] != "": 
                continue
            
                
            data[k].append(item)
            
        

    item0 = {"t": 1579582800, "dailyMortality": 1, "dailyCases": 1,
                "caseRateMean": 0, "mortalityMean": 0,
                "hospDaily": 0,
                "percent14dayDailyCases" : 0, "percent14dayDailyDeaths": 0}
    item1 = {"t": 1588731173, "dailyMortality": 1, "dailyCases": 1,
                "caseRateMean": 0, "mortalityMean": 0,
                "hospDaily": 0,
                "percent14dayDailyCases" : 0, "percent14dayDailyDeaths": 0}
    
    itemNA = {"t": 1585713600,
            "dailyMortality": 1, "dailyCases": 1,
                "caseRateMean": 0, "mortalityMean": 0,
                "hospDaily": 0,
                "percent14dayDailyCases" : 0, "percent14dayDailyDeaths": 0}
    
    data["99999"] = [itemNA]

    data["_"] = [item0, item1]

    output = defaultdict(dict)
    for k, v in data.items():
        if k == "_nation":
            continue
        output[k[:2]] = v

    with open(f"../timeseriesAll.json", "w") as fp:
            json.dump(output, fp, indent=2)
    



if __name__=="__main__":

    linechart()